In [81]:
import pydeck as pdk
import pandas as pd


In [82]:
data=pd.read_csv('data.csv')
data.head( )

,time,latitude,longitude,altitude,speed,speed_elevation
0,2024-02-07 12:21:17+00:00,45.310150,5.891150,980.0,NaN,NaN
1,2024-02-07 12:21:18+00:00,45.310117,5.891200,978.0,5.728637,-2.0
2,2024-02-07 12:21:19+00:00,45.310083,5.891217,973.0,6.410651,-5.0
3,2024-02-07 12:21:20+00:00,45.310050,5.891233,969.0,5.573503,-4.0
4,2024-02-07 12:21:21+00:00,45.310000,5.891250,966.0,6.461528,-3.0


In [83]:
viewport = pdk.data_utils.compute_view(points=data[['longitude', 'latitude']], view_proportion=0.9)


In [84]:
import numpy as np

# Fonction pour calculer la moyenne glissante des taux de montée
def moving_average(data, window_size):
    window_size = 15
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Fonction pour détecter les thermiques
def detect_thermals(data, threshold_altitude, threshold_climb_rate):
    window_size = 15
    avg_climb_rates = moving_average(data["speed_elevation"], window_size)
    thermals = []
    coordinates_thermal_latitude =[]
    coordinates_thermal_longitude =[]
    coordinates_thermal_altitude =[]
    in_thermal = False
    start_idx = 0
    thermics=[]

    for i in range(1+window_size, len(data["altitude"])-1):
        if data["altitude"][i] > threshold_altitude and avg_climb_rates[i-(window_size)] > threshold_climb_rate:
            if not in_thermal:
                in_thermal = True
                start_idx = i 
                coordinates_thermal_latitude.append(data["latitude"][i])
                coordinates_thermal_longitude.append(data["longitude"][i])
                coordinates_thermal_altitude.append(data["altitude"][i])
            
                
            if in_thermal:
                coordinates_thermal_latitude.append(data["latitude"][i])
                coordinates_thermal_longitude.append(data["longitude"][i])
                coordinates_thermal_altitude.append(data["altitude"][i])
             
                            
        else:
            if in_thermal and i-start_idx>50:
                in_thermal = False
                end_idx = i
                thermals.append((start_idx, end_idx,coordinates_thermal_latitude,coordinates_thermal_longitude,coordinates_thermal_altitude))
                coordinates_thermal_longitude =[]
                coordinates_thermal_latitude =[]
                coordinates_thermal_altitude =[]

            if in_thermal and i-start_idx<=50:
                in_thermal = False
                coordinates_thermal_longitude =[]
                coordinates_thermal_latitude =[]
                coordinates_thermal_altitude=[]
             
    return thermals


# Seuils pour la détection de thermiques (altitude minimale et taux de montée minimal)
threshold_altitude = 130
threshold_climb_rate = 0.5

# Détection des thermiques
thermals = detect_thermals(data, threshold_altitude, threshold_climb_rate)


# Affichage des résultats
for thermal in thermals:
    start_idx, end_idx, latitude,longitude,altitude = thermal
    print("Thermal detected from index", start_idx, "to", end_idx)
    
list_columns=["Start","End","Lat","Long","Altitude"]
df=pd.DataFrame(thermals,columns=list_columns)
df.to_csv('thermal.csv')

# Concatenate all latitude, longitude, and altitude lists into a single list
all_latitude = [lat for thermal in thermals for lat in thermal[2]]
all_longitude = [long for thermal in thermals for long in thermal[3]]
all_altitude = [alt for thermal in thermals for alt in thermal[4]]

# Create a DataFrame from the concatenated lists
data_thermal = pd.DataFrame({
    "Latitude": all_latitude,
    "Longitude": all_longitude,
    "Altitude": all_altitude
})


Thermal detected from index 851 to 978
Thermal detected from index 1094 to 1158
Thermal detected from index 2898 to 2964
Thermal detected from index 3000 to 3171
Thermal detected from index 4812 to 4966
Thermal detected from index 5293 to 5392


In [85]:
data_thermal

,Latitude,Longitude,Altitude
0,45.284833,5.859600,1038.0
1,45.284833,5.859600,1038.0
2,45.284767,5.859583,1039.0
3,45.284717,5.859617,1040.0
4,45.284667,5.859667,1042.0
...,...,...,...
682,45.288717,5.835450,1467.0
683,45.288750,5.835367,1468.0
684,45.288800,5.835300,1469.0
685,45.288867,5.835250,1471.0


In [86]:
#data_thermal['Altitude']=data_thermal['Altitude']-1000

In [87]:
import pydeck as pdk

# Set your Mapbox API key
MAPBOX_API_KEY = "pk.eyJ1IjoiZ3dlbjk2IiwiYSI6ImNsdHZyMDlwMjAwMTgyamxtbzBtY2hwM20ifQ.6ALyS2q426nnberd9NXUaA"

# Create a scatterplot layer
scatterplot = pdk.Layer(
    'ScatterplotLayer',
    data_thermal,
    get_position=['Longitude', 'Latitude',"Altitude"],
    auto_highlight=True,
    pickable=True,
    extruded=True,
    coverage=1,
    radius_scale=8,
    get_color=[255, 0, 0]
)
scatterplot2 = pdk.Layer(
    'ScatterplotLayer',
    data,
    get_position=['longitude', 'latitude','altitude'],
    auto_highlight=True,
    pickable=True,
    extruded=True,
    coverage=1,
    radius_scale=3,
    get_color=[0, 0, 255])

# Define viewport
viewport = pdk.data_utils.compute_view(data_thermal[['Longitude', 'Latitude', 'Altitude']])

# Create Deck with Mapbox background
r = pdk.Deck(
    layers=[scatterplot,scatterplot2],
    initial_view_state=viewport,
    mapbox_key= MAPBOX_API_KEY,  # Pass your Mapbox API key using mapbox_key parameter
    map_style='mapbox://styles/mapbox/satellite-v9',
    
)

r.to_html("a.html")



In [88]:
data

,time,latitude,longitude,altitude,speed,speed_elevation
0,2024-02-07 12:21:17+00:00,45.310150,5.891150,980.0,NaN,NaN
1,2024-02-07 12:21:18+00:00,45.310117,5.891200,978.0,5.728637,-2.0
2,2024-02-07 12:21:19+00:00,45.310083,5.891217,973.0,6.410651,-5.0
3,2024-02-07 12:21:20+00:00,45.310050,5.891233,969.0,5.573503,-4.0
4,2024-02-07 12:21:21+00:00,45.310000,5.891250,966.0,6.461528,-3.0
...,...,...,...,...,...,...
6669,2024-02-07 14:12:42+00:00,45.265767,5.841600,463.0,0.000000,0.0
6670,2024-02-07 14:12:43+00:00,45.265767,5.841600,463.0,0.000000,0.0
6671,2024-02-07 14:12:44+00:00,45.265767,5.841600,463.0,0.000000,0.0
6672,2024-02-07 14:12:45+00:00,45.265767,5.841600,463.0,0.000000,0.0
